In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Weather Data
First do this on the HQ zipcodes.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags.csv")

Do this across all zips, for the establishment records. We'll put this into a different format right after, and then change the columns and whatnot.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_allZips.csv").\
    drop(columns = {"Unnamed: 0", 'Unnamed: 0.1'})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags_allZips.csv")

Add in the few new definitions we've started on here: bins by week/month/quarter, and days 90+.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/latestExtremes.csv").\
    drop(columns = {"Unnamed: 0"})
allWeather.rename(columns = {'quarter': 'qtr'}, inplace = True)

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags_new.csv")

Now do the same for the industry-specific weather.

In [ ]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\
allWeather_byInd = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_byInd.csv").\
    drop(columns = {"Unnamed: 0"})
'''[['famafrench','zipcode','yearQuarter', 
                                    'temp_ffquant_0.95','temp_indQuarterquant_0.95',
                                   'temp5Days_ffquant_0.95', 'temp5Days_indQuarterquant_0.95',
                                   'precip_ffquant_0.95', 'precip_indQuarterquant_0.95',
                                   'precip5Days_ffquant_0.95', 'precip5Days_indQuarterquant_0.95']]
'''
allWeather_byInd['year'] = allWeather_byInd.yearQuarter.str.slice(0,4).astype('int64')
allWeather_byInd['qtr']  = allWeather_byInd.yearQuarter.str.slice(5,6).astype('int64')
allWeather_byInd['yearQtr'] = allWeather_byInd.year + (allWeather_byInd.qtr - 1)/4

allWeather_byInd = allWeather_byInd.astype({'year':       'category',
                         'qtr':        'category',
                         'zipcode':    'category',
                         'famafrench': 'category'})

changes['zipcode'] = changes['zipcode'].astype({'zipcode': 'int64'})

changes = changes.astype({'year':       'category',
                          'qtr':        'category',
                          'zipcode':    'category',
                          'famafrench': 'category'})

col = allWeather_byInd.pop("year")
allWeather_byInd.insert(0, col.name, col)

col = allWeather_byInd.pop("qtr")
allWeather_byInd.insert(0, col.name, col)


col = allWeather_byInd.pop("yearQtr")
allWeather_byInd.insert(0, col.name, col)

allWeather_byInd.drop(columns = {'yearQuarter'}, inplace = True)

print(allWeather_byInd.head())

In [ ]:
lag1 = allWeather_byInd.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[5:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)
lag1 = lag1.astype({'yearQtr':       'category'})

    
lag2 = allWeather_byInd.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[5:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)
lag2 = lag2.astype({'yearQtr':       'category'})


lag3 = allWeather_byInd.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[5:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)
lag3 = lag3.astype({'yearQtr':       'category'})


lag4 = allWeather_byInd.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[5:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)
lag4 = lag4.astype({'yearQtr':       'category'})


allWeather_byInd = allWeather_byInd.astype({'yearQtr':       'category'})


print(allWeather_byInd.shape)


allWeather_byInd.head()


'''allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

allWeather_byInd_withLags.year.value_counts()
'''

In [ ]:
allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

In [ ]:
allWeather_byInd_withLags.to_csv("../../data/companyData/allWeather_byInd_withLags.csv")

In [ ]:
allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv")
allWeather_byInd_withLags.head()

In [ ]:
del allWeather_byInd
del lag1
del lag2
del lag3
del lag4
gc.collect()

Now get the streak data. Make sure it's 0/1 for whether there was a streak.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/stockWeather_zipQuarterQuants.csv").\
    drop(columns = {'Unnamed: 0'})

allWeather = allWeather[~allWeather.temp_zipQuarterLast5.isna()].reset_index(drop = True)

allWeather['date'] = pd.to_datetime(allWeather['date'],
                                   format = "%Y-%m-%d")

allWeather['hotStreak']   = (allWeather['temp_zipQuarterLast5'] == 5)*1
allWeather['wetStreak'] = (allWeather['precip_zipQuarterLast5'] == 5)*1

allWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(allWeather.dtypes)

allWeather['year'] = allWeather.date.dt.year
allWeather['quarter'] = allWeather.date.dt.quarter


allWeather.drop(columns = {'date','temp_zipQuarterLast5','precip_zipQuarterLast5'}, inplace = True)

streaks = allWeather.groupby(['zipcode','year','quarter']).sum().reset_index()

streaks['hotStreak'] = (streaks['hotStreak'] > 0)*1
streaks['wetStreak'] = (streaks['wetStreak'] > 0)*1

streaks.rename(columns = {'quarter': 'qtr'}, inplace = True)
streaks.head()


In [ ]:
streaks['yearQtr'] = streaks.year + (streaks.qtr - 1)/4

col = streaks.pop("yearQtr")
streaks.insert(0, col.name, col)

lag1 = streaks.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = streaks.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = streaks.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = streaks.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(streaks.shape)

streaks_withLags = streaks.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(streaks_withLags.year.value_counts())

streaks_withLags.to_csv("../../data/companyData/streaks_withLags.csv")

In [ ]:
streaks_withLags = pd.read_csv("../../data/companyData/streaks_withLags.csv")
streaks_withLags.year.value_counts()

In [ ]:
streaks_withLags.head()

And now the severe weather.

In [ ]:
zipQuarters = pd.read_csv("../../data/companyData/allWeather_withLags_new.csv")[['zipcode','year','qtr']].drop_duplicates()


zipQuarters_2009 = zipQuarters.copy()
zipQuarters_2009 = zipQuarters[zipQuarters.year == 2010]
zipQuarters_2009['year'] = zipQuarters_2009['year'] - 1

zipQuarters = zipQuarters_2009.append(zipQuarters)


zipQuarters.head()

In [ ]:
zipQuarters.year.min()

In [ ]:
thunderstorms = pd.read_csv("../../data/companyData/thunderstormWinds.csv").\
    drop(columns = {'Unnamed: 0', 'STCOUNTYFP'})

thunderstorms['year'] =  thunderstorms['yearQtr'].str.slice(0,4).astype('int64')

print(thunderstorms.year.min())

thunderstorms.head()


thunderstorms['qtr']  =  thunderstorms['yearQtr'].str.slice(5,6).astype('int64')
thunderstorms.drop(columns = {'yearQtr'}, inplace = True)
thunderstorms = thunderstorms[(thunderstorms.year > 2008) & (thunderstorms.year < 2020)]

print(thunderstorms.shape)

thunderstorms = zipQuarters.merge(thunderstorms, how = 'left')
thunderstorms = thunderstorms.fillna(0)
print(zipQuarters.shape, thunderstorms.shape)

thunderstorms.year.min()

In [ ]:
thunderstorms = thunderstorms.groupby(['zipcode','year','qtr']).sum().reset_index()

thunderstorms['propAboveTenThou']     = (thunderstorms['propAboveTenThou'] > 0)*1
thunderstorms['propAboveHundredThou'] = (thunderstorms['propAboveHundredThou'] > 0)*1
thunderstorms['propAboveMilli']       = (thunderstorms['propAboveMilli'] > 0)*1


thunderstorms.year.value_counts()

In [ ]:
thunderstorms['yearQtr'] = thunderstorms.year + (thunderstorms.qtr - 1)/4

col = thunderstorms.pop("year")
thunderstorms.insert(0, col.name, col)

col = thunderstorms.pop("qtr")
thunderstorms.insert(0, col.name, col)

col = thunderstorms.pop("yearQtr")
thunderstorms.insert(0, col.name, col)

col = thunderstorms.pop("zipcode")
thunderstorms.insert(0, col.name, col)

thunderstorms = thunderstorms.astype({'zipcode':    'category'})


lag1 = thunderstorms.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)
lag1 = lag1.astype({'yearQtr':    'category'})



lag2 = thunderstorms.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)
lag2 = lag2.astype({'yearQtr':    'category'})


lag3 = thunderstorms.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)
lag3 = lag3.astype({'yearQtr':    'category'})

lag4 = thunderstorms.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)
lag4 = lag4.astype({'yearQtr':    'category'})


thunderstorms = thunderstorms.astype({'yearQtr':    'category'})

thunderstorms_withLags = thunderstorms.merge(lag1).merge(lag2).merge(lag3).merge(lag4)
print(thunderstorms_withLags.year.value_counts())

thunderstorms_withLags.to_csv("../../data/companyData/thunderstorms_withLags.csv")

In [ ]:
print(thunderstorms_withLags.head())

# Locations
Create a separate definition of weather based not on HQ but on employee-weighted establishment footprint.

In [ ]:
fractions = pd.read_csv('../../data/companyData/fractionEmployees_byEstablishment.csv').\
    drop(columns = {"Unnamed: 0", 'latitude','longitude'}).rename(columns = {'archive_version_year': 'year',
                                                    'parent_number': 'abi'})

fractions['year']    = fractions.year.astype('int64')
fractions['zipcode'] = fractions.zipcode.astype('int64')
fractions.head()

gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

print(gvKey_abiLinkingTable.abi)

gvKey_abiLinkingTable.head()

fractions = fractions[['year','abi','zipcode','locationFracOfEmployees']].merge(gvKey_abiLinkingTable[['abi','gvkey']])

fractions = fractions.astype({'year':       'category',
                           'zipcode':    'category'})

fractions.head()

In [ ]:
fractions.year

In [ ]:
fractions_byZip = fractions[['gvkey','year','zipcode','locationFracOfEmployees']]
fractions_byZip['year'] =  fractions_byZip['year'].astype('int64')

fractions_byZip = fractions_byZip[(fractions_byZip.year > 2008) & (fractions_byZip.year < 2020)]

print(fractions_byZip.shape)

fractions_byZip.to_csv("../../data/companyData/fractions_byZip.csv")

In [ ]:
fractionsWithWeather = fractions.merge(allWeather_withLags_allZips) 
fractionsWithWeather.drop(columns = {'abi','zipcode'}, inplace = True)

print(fractionsWithWeather.shape)
fractionsWithWeather.head()

In [ ]:
fractionsWithWeather[fractionsWithWeather.gvkey == 1004]

In [ ]:
del allWeather_withLags
del fractions
del gvKey_abiLinkingTable
gc.collect()

In [ ]:
for col in fractionsWithWeather.columns[4:]:
    fractionsWithWeather[col] = fractionsWithWeather[col] * fractionsWithWeather.locationFracOfEmployees

In [ ]:
g = fractionsWithWeather.groupby(['gvkey','year','qtr']).sum().reset_index()
g.drop(columns = {'locationFracOfEmployees'}, inplace = True)

for colname in g.columns[3:]:
    g.rename(columns = {colname: 'empWt_' + colname}, inplace = True)

g.head()

In [ ]:
g.to_csv("../../data/companyData/weatherByEstablishment.csv")

In [ ]:
establishmentZips = fractions.zipcode.unique()
len(establishmentZips)